In [ ]:
# Set this variable yourself.
running_on_colab = False
# Store data as reduced density matrix `rho` or eigenvector tuple `EVW`.
rho_or_EVW = 'rho'

# Machine Learning of Many Body Localization

## Introduction

Use exact diagonalization to obtain a few eigenstates near energy $E = 0$ from the Heisenberg model with a
random field, 

\begin{equation}
    H = J \sum_i \vec{S}_{i} \cdot \vec{S}_{i+1} - \sum_i h_i S^z_i
\end{equation}

, where the values of the field $ h_i \in [-W, W] $ are chosen from a uniform random distribution with a "disorder strength" $W$ (with moderate system sizes $L \approx 12$). 

The exciting property of this model is that it is believed to undergo a phase transition from an extended phase (small $W$) to a localized phase (large $W$). 

We will use ML to detect this transition: Pick a number of eigenstates that are near energy $E = 0$ and obtain the reduced density matrices $\rho^A$, where $A$ is a region of $n$ consecutive spins (a few hundred to thousands eigenstates for different disorder realizations). 

Now use the density matrices for $W = 0.5 J$ and $W = 8.0 J$ to train a neural network (just interpret the entries of $\rho^A$ as an image with $2^n \times 2^n$ pixel). 
Then use this network and study the output of the neural network for different $W$. 

How does the results depend on system size $L$ and block size $n$? 
At which $W_c$ do you expect the transition to occur?

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

os.environ['running_on_colab'] = str(running_on_colab)
# running_on_colab = (os.getenv('running_on_colab', 'False') == 'True')

if running_on_colab:
    data_root             = 'drive/MyDrive/Colab Data/MBL/'
    sys.path.append(data_root)
else:
    data_root             = './'

# Store data as reduced density matrix `rho` or eigenvector tuple `EVW`.
os.environ['rho_or_EVW'] = str(rho_or_EVW)
# running_on_colab = (os.getenv('rho_or_EVW', 'EVW') == 'rho')

import warnings
warnings.filterwarnings('ignore')

from file_io import *
from data_gen import *

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import MaxNLocator

dpi = 100
fig_w = 640
fig_h = 480

%matplotlib inline

In [ ]:
if running_on_colab:
    !cat /proc/cpuinfo

In [ ]:
if running_on_colab:
    !pip install ipython-autotime
    %load_ext autotime

In [ ]:
if running_on_colab:
    !pip install pytorch_lightning==0.7.6 torchsummary==1.5.1

## Plot reduced density matrix
Plot the magnitude of $ 4 \times 4 $ typical reduced density matrices.  
$L = 12, n = 6, k = 1$ is selected.  
Actually $L = 10, n = 6, k = 5$ now.

In [ ]:
MBL = {
    "obj_name": 'rho_A',
    "L": 10,
    "n": 6,
    "periodic": True,
    "num_EV": 5,
    "rho_train_data_dir": rho_train_data_dir,
    "rho_valid_data_dir": rho_valid_data_dir
}
obj_name = MBL['obj_name']
L        = MBL['L']
n        = MBL['n']
periodic = MBL['periodic']
p        = MBL['periodic']
num_EV   = MBL['num_EV']

In [ ]:
from MBL_dataset import MBLDataset

data_train = load_rho_train(obj_name, L, n, periodic, num_EV, data_dir=rho_train_data_dir)
data_valid = load_rho_valid(obj_name, L, n, periodic, num_EV, data_dir=rho_valid_data_dir)

train_dataset = MBLDataset(
    data=data_train,
    train=True,
    transform=transforms.ToTensor(),
)
valid_dataset = MBLDataset(
    data=data_valid,
    train=False,
    transform=transforms.ToTensor(),
)

print('Number of training samples  :', len(train_dataset))
print('Number of validation samples:', len(valid_dataset))

In [ ]:
# Two classes.
labels = ['Extended (Low W)', 'Localized (High W)']

image, W, label = train_dataset[0]["image"], train_dataset[0]["W"], train_dataset[0]["label"]
print("W: {:.2f}\nLabel: {}".format(W, labels[label]))
print("Shape of the image:", image.size())
print("Smallest value in the image:", torch.min(image))
print("Largest value in the image:", torch.max(image))
# print(image)

In [ ]:
def visualize_train_data(dataset):
    
    num = 4

    sample_idx = np.random.randint(0, len(dataset), size=num*num)

    # Square image.
    fig, axes = plt.subplots(num, num, figsize=(fig_w/dpi,fig_w/dpi), dpi=dpi, squeeze=False)

    samples_ext = []
    samples_loc = []
    for i in range(len(dataset)):
        data = dataset[i]
        if data['W'] == 0.5 and len(samples_ext) < num*num//2:
            samples_ext.append(np.abs(data['image'].squeeze(axis=0).numpy()))
        if data['W'] == 8.0 and len(samples_loc) < num*num//2:
            samples_loc.append(np.abs(data['image'].squeeze(axis=0).numpy()))
        if len(samples_ext) >= num*num//2 and len(samples_loc) >= num*num//2:
            break

    i_ext = 0
    i_loc = 0
    for i in range(num * num):
        if i // num < 2:
            # np.fill_diagonal(samples_ext[i_ext], 0)
            axes[i%num,i//num].imshow(samples_ext[i_ext])
            axes[i%num,i//num].annotate('W={:3.1f}'.format(0.5), (0.25,0.1), xycoords='axes fraction', ha='center', color='w')
            i_ext += 1
        else:
            # np.fill_diagonal(samples_loc[i_loc], 0)
            axes[i%num,i//num].imshow(samples_loc[i_loc])
            axes[i%num,i//num].annotate('W={:3.1f}'.format(8.0), (0.25,0.1), xycoords='axes fraction', ha='center', color='w')
            i_loc += 1

    for axe in axes:
        for ax in axe:
            # ax.legend(loc='best')
            ax.xaxis.set_ticklabels([])
            ax.yaxis.set_ticklabels([])
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False)

    fig.tight_layout()

In [ ]:
print('Visualize training data:')
visualize_train_data(train_dataset)

## Dumb counter instead of CNN
  
Using $L = 10, n = 6, k = 5$ now.

In [ ]:
MBL = {
    "obj_name": 'rho_A',
    "L": 10,
    "n": 6,
    "periodic": True,
    "num_EV": 5,
    "rho_train_data_dir": rho_train_data_dir,
    "rho_valid_data_dir": rho_valid_data_dir
}
obj_name = MBL['obj_name']
L        = MBL['L']
n        = MBL['n']
periodic = MBL['periodic']
p        = MBL['periodic']
num_EV   = MBL['num_EV']

In [ ]:
# Count terms
def count_offdiagonal(dataset):
    
    off_ext = {}
    off_loc = {}
    size = 0

    for i in tqdm(range(len(dataset))):

        data  = dataset[i]
        W     = data['W']
        image = np.abs(data['image'].squeeze(axis=0).numpy())
        np.fill_diagonal(image, 0)

        if i == 0:
            size = image.size

        for exp in range(33):

            if exp not in off_ext:
                off_ext[exp] = []
            if exp not in off_loc:
                off_loc[exp] = []

            count = (image <= 10**(-exp)).sum()

            if W == 0.5:
                off_ext[exp].append(count)
            if W == 8.0:
                off_loc[exp].append(count)

    for (exp_ext, counts_ext), (exp_loc, counts_loc) in zip(off_ext.items(), off_loc.items()):
        print('Avg #elements below 1e-{: <2}. Ext: {: >7.2f} | Loc: {: >7.2f} | Diff: {: >+8.2f}'.format(
            exp_ext, np.mean(counts_ext), np.mean(counts_loc), np.mean(counts_ext) - np.mean(counts_loc)
        ))

    print(' ')

    for (exp_ext, counts_ext), (exp_loc, counts_loc) in zip(off_ext.items(), off_loc.items()):
        print('Avg #elements below 1e-{: <2}. Ext: {: >6.2f}% | Loc: {: >6.2f}% | Diff: {: >+7.2f}%'.format(
            exp_ext, np.mean(counts_ext)/size*100, np.mean(counts_loc)/size*100, (np.mean(counts_ext) - np.mean(counts_loc))/size*100
        ))



In [ ]:
count_offdiagonal(train_dataset)

In [ ]:
def calc_probs_counter(dataset):

    # Sample model predictions.
    result_images  = []
    result_targets = []
    result_Ws      = []
    result_preds   = []
    result_probs   = []
    size = 0
    temp = []
    
    for i in tqdm(range(len(dataset))):

        data  = dataset[i]
        W     = data['W']
        image = np.abs(data['image'].squeeze(axis=0).numpy())
        np.fill_diagonal(image, 0)

        if i == 0:
            size = image.size

        ratio = (image <= 1e-18).mean()
        # ratio = (ratio - 0.2974)/.4385
        temp.append(ratio)

        if ratio < 0.5:
            Ps = [1, 0]
        else:
            Ps = [0, 1]
        # Ps = [1-ratio, ratio]

        # result_Ws      = result_Ws      + [W]
        # result_probs   = result_probs   + [Ps]
        result_Ws.append(W)
        result_probs.append(Ps)

    # print(np.mean(temp))
    # print(np.min(temp))
    # print(np.max(temp))

    result_Ws    = np.array(result_Ws)
    result_probs = np.array(result_probs)
    sorted_idx   = result_Ws.argsort()
    Ws = result_Ws[sorted_idx]
    Ps = result_probs[sorted_idx]

    # Compute mean and std.
    Ws_dict = OrderedDict()
    Ws_uniq = []
    Ps_mean = []
    Ps_std  = []
    # Ws is already sorted in `calc_probs()`.
    for W, P in zip(Ws, Ps[:,1]):
        if W not in Ws_dict:
            Ws_dict[W] = []
        Ws_dict[W].append(P)
    for (W, P) in Ws_dict.items():
        Ws_uniq.append(W)
        Ps_mean.append(np.mean(P))
        Ps_std.append(np.std(P, ddof=1))

    return Ws, Ps, np.array(Ws_uniq), np.array(Ps_mean), np.array(Ps_std)


In [ ]:
# Remove y0 because it should be bounded/aligned with y = 0 and y = 1.
def sigmoid(x, x0, y0, b, C):
    y = C / (1 + np.exp(-b * (x - x0))) + y0
    return y

# Logit function is the inverse of sigmoid.
def logit(y, x0, y0, b, C):
    x = np.log((y - y0) / (C - (y - y0))) / b + x0
    return x


In [ ]:
def plot_counter_crossing(Ws, Ps, Ws_uniq, Ps_mean, Ps_std, L, n, periodic):

    labels = ['Extended (Low W)', 'Localized (High W)']

    # Plot probability P(Localized) against W.
    # 4:3 aspect ratio.
    fig, axes = plt.subplots(1, 1, figsize=(fig_w/dpi, fig_h/dpi), dpi=dpi, squeeze=False)

    fig.suptitle('Probability vs Disorder Strength ($L={}, n={}, {}$)'.format(L, n, 'periodic' if periodic else 'non-periodic'))

    axes[0,0].axhline(0.5, c='lightgrey', ls='--', label='$P=0.5$')

    # Plot averaged values with error bars.
    # markers, caps, bars = axes[0,0].errorbar(Ws_uniq, Ps_mean, Ps_std, ls=' ', marker='x', capsize=2, capthick=2, label='P(Localized) Mean')
    # Loop through bars and caps and set the alpha value
    # [bar.set_alpha(0.5) for bar in bars]
    # [cap.set_alpha(0.5) for cap in caps]
    
    # Actually, don't plot error bars. That's distracting.
    axes[0,0].plot(Ws_uniq, Ps_mean, ls=' ', marker='x', c='#0065bd', label='$E_W[P(Localized)]$')

    # Plot raw data.
    axes[0,0].plot(Ws, Ps,   ls=' ', marker='.', c='#98c6ea', label='$P(Localized)$', alpha=0.05)
    axes[0,0].set_xlabel('Disorder strength $W$')
    axes[0,0].set_ylabel('Probability $P(Localized)$')

    # Curve fit a sigmoid using all data.
    # Fitting only the mean with `Ws_uniq` and `Ps_mean` gives identical results.
    # popt, pcov = curve_fit(sigmoid, Ws, Ps, p0=[3, 0, 2]) # Add bounds or initial values if it doesn't converge.
    popt, pcov = curve_fit(sigmoid, Ws, Ps, p0=[3, 0, 2, 1]) # Add bounds or initial values if it doesn't converge.
    # x0, y0, b = popt
    x0, y0, b, C = popt
    x = np.linspace(0, 10, 100)
    y = sigmoid(x, *popt)
    axes[0,0].plot(x, y, ls='--', lw=2, c='#e37222', label='Fited sigmoid')
    axes[0,0].set_title('$\sigma(x) = {:.4f} / (1 + Exp(-{:.4f} (x - {:.4f}))) + {:.4f}$'.format(C, b, x0, y0), fontsize=10)
    print('Fitted sigmoid function {:.4f} / (1 + Exp(-{:.4f} (x - {:.4f}))) + {:.4f}'.format(C, b, x0, y0))
    # print('Fitted sigmoid function 1 / (1 + Exp(-{:.4f} (x - {:.4f}))) + {:.4f}'.format(b, x0, y0))

    W_c = logit(0.5, *popt)
    perr = np.sqrt(np.diag(pcov))
    # The error is actually wrong here. Should have put it through logit.
    print('Transition W_C is found to be at W = {:.4f} ± {:.4f}'.format(W_c, perr[0]))
    axes[0,0].axvline(W_c, c='r',         ls='--', label='Fitted $W_c$')

    for axe in axes:
        for ax in axe:
            ax.tick_params(direction="in")
            ax.legend(loc='best')
            ax.set_xlim(0, 6)
            # ax.set_ylim(-0.2,1.2)


In [ ]:
valid_Ws, valid_Ps, valid_Ws_uniq, valid_Ps_mean, valid_Ps_std = calc_probs_counter(valid_dataset)

In [ ]:
plot_counter_crossing(valid_Ws, valid_Ps[:,1], valid_Ws_uniq, valid_Ps_mean, valid_Ps_std, MBL['L'], MBL['n'], MBL['periodic'])

In [ ]:
del data_train
del data_valid
del train_dataset
del valid_dataset

## Neural network

Since NNs with the same `n` have the same input size, we will evaluate them using the same NN structure. As a side effect, results different `n` are not entirely comparable, but we will compare them anyway because reasons.  

Two classes `MBLModel` and `MBLDataset`, modified from a previous CNN facial recoginition code (own work), are used. The model structure and hyperparameters are defined using a dict called `hparams`. Inside it, specifications of the training data are passed using a nested dict `hparams["MBL"]`. The models are stored in a directory structure that mirrors that of the training data (reduced density matrices $\rho_A$).  

Caveat: Validation data isn't really unseen data from the training distribution $W \in \{0.5, 8\}$, but rather random W's that we'll be using them to predict $W_c$.  

See the other notebook for data generation.  

In [ ]:
MBL = {
    "obj_name": 'rho_A',
    "L": 10,
    "n": 6,
    "periodic": True,
    "num_EV": 5,
    "rho_train_data_dir": rho_train_data_dir,
    "rho_valid_data_dir": rho_valid_data_dir
}
obj_name = MBL['obj_name']
L        = MBL['L']
n        = MBL['n']
periodic = MBL['periodic']
p        = MBL['periodic']
num_EV   = MBL['num_EV']

In [ ]:
from MBL_model import MBLModel
model_version = 1


In [ ]:
model = load_model('model_v{}.pkl.gz'.format(model_version), L, n, periodic, num_EV)
# model = load_H_model('model_v{}.pkl.gz'.format(model_version), L, periodic)

### Visualize model predictions

In [ ]:
def visualize_predictions(model, mode='train'):
    """Mode = ['train' | 'valid']"""

    labels = ['Extended (Low W)', 'Localized (High W)']

    num = 4

    # Sample model predictions.
    result_images  = []
    result_targets = []
    result_Ws      = []
    result_preds   = []
    result_probs   = []

    model.eval()
    SM  = torch.nn.Softmax()
    LSM = torch.nn.LogSoftmax()
    if mode == 'train':
        dataloader = DataLoader(model.dataset["train"], batch_size=num**2, shuffle=True)#, pin_memory=True)
    else:
        dataloader = DataLoader(model.dataset["val"], batch_size=num**2, shuffle=True)#, pin_memory=True)

    for batch in dataloader:

        images, targets, Ws = batch["image"], batch["label"], batch["W"]
        images  = images.to(device)
        outputs = model(images)
        images  = images.to('cpu')
        outputs = outputs.to('cpu')

        preds   = outputs.argmax(axis=1)
        probs   = SM(outputs)
        # probs2  = - LSM(outputs)
        # out_sum = probs2[:,0] + probs2[:,1]
        # probs2[:,0] = probs2[:,0] / out_sum
        # probs2[:,1] = probs2[:,1] / out_sum
        # Simple averaging doesn't work, because it's negative...
        # out_sum = outputs[:,0] + outputs[:,1]
        # outputs[:,0] = outputs[:,0] / out_sum
        # outputs[:,1] = outputs[:,1] / out_sum
        shape   = images.shape
        result_images  = result_images  + images.reshape(shape[0], shape[2], shape[3]).tolist()
        result_targets = result_targets + targets.tolist()
        result_Ws      = result_Ws      + Ws.tolist()
        result_preds   = result_preds   + preds.tolist()
        result_probs   = result_probs   + probs.tolist()
        # result_probs   = result_probs   + probs2.tolist()
        break # Because we only need 25 images.

    # Display images.
    sample_idx = np.random.randint(0, len(result_preds), size=num**2)

    # Square image.
    fig, axes = plt.subplots(num, num, figsize=(fig_w/dpi,fig_w/dpi), dpi=dpi, squeeze=False)

    for i, idx in enumerate(sample_idx):
        W      = result_Ws[idx]
        W_in   = result_targets[idx]
        W_pred = result_preds[idx]
        W_prob = result_probs[idx]
        if W_in == W_pred:
            ec = 'lime'
            tc = 'white'
        else:
            ec = 'red'
            tc = 'red'

        axes[i%num,i//num].imshow(np.abs(result_images[idx]))

        axes[i%num,i//num].tick_params(color=ec, labelcolor=ec)
        for spine in axes[i%num,i//num].spines.values():
            spine.set_edgecolor(ec)
            spine.set_linewidth(2)

        annotation  = 'Input  : \n{}\nW={:.2f}\n\n'.format(labels[W_in], W)
        annotation += 'Predict: \n{}\n{:.0f}%'.format(labels[W_pred], W_prob[W_pred]*100)
        # annotation += 'Predict: \n{}\n{:.0f}%'.format(labels[W_pred], W_prob[(W_pred+1)%2]*100)
        # axes[i%num,i//num].annotate(annotation, (0.5,0.275), xycoords='axes fraction', ha='center', color='w', bbox=dict(facecolor='none', edgecolor=ec, boxstyle='round,pad=1', linewidth=2))
        axes[i%num,i//num].annotate(annotation, (0.5,0.13), xycoords='axes fraction', ha='center', color=tc, fontsize=9)#, bbox=dict(facecolor='none', edgecolor=ec, boxstyle='round,pad=0.5', linewidth=1))

    for axe in axes:
        for ax in axe:
            # ax.legend(loc='best')
            ax.xaxis.set_ticklabels([])
            ax.yaxis.set_ticklabels([])
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False)

    fig.tight_layout()


### Sample training data

In [ ]:
visualize_predictions(model, 'train')

### Sample validation data

In [ ]:
visualize_predictions(model, 'valid')

### Model performance

In [ ]:
def evaluate_model_core(model, dataset):

    model.eval()
    criterion = torch.nn.CrossEntropyLoss()
    SM = torch.nn.Softmax()
    dataloader = DataLoader(dataset, batch_size=1000, shuffle=False)#, pin_memory=True)
    loss = 0
    n_correct = 0

    for batch in dataloader:
        images, targets = batch["image"], batch["label"]
        images  = images.to(device)
        outputs = model(images).to('cpu')
        preds   = outputs.argmax(axis=1)
        # print(SM(outputs))
        loss += criterion(outputs, targets).item()
        n_correct += (preds == targets).sum().item()

    return loss, n_correct / len(dataset)

def evaluate_model(model):

    print("Training accuracy  : {:.4f}%".format(evaluate_model_core(model, model.dataset["train"])[1] * 100))
    print("Validation accuracy: {:.4f}%".format(evaluate_model_core(model, model.dataset["val"])[1]   * 100))


In [ ]:
evaluate_model(model)

In [ ]:
del model.dataset["train"]
del model.dataset["val"]
del model

## Plot curve fitting (Estimate transition disorder strength)
Plot sigmoid curve fitting of transition disorder strength $W_c$ with error.  
$L = 12, n = 6, k = 1$ is selected.    
Actually $L = 10, n = 6, k = 5$ now.

In [ ]:
MBL = {
    "obj_name": 'rho_A',
    "L": 10,
    "n": 6,
    "periodic": True,
    "num_EV": 5,
    "rho_train_data_dir": rho_train_data_dir,
    "rho_valid_data_dir": rho_valid_data_dir
}
obj_name = MBL['obj_name']
L        = MBL['L']
n        = MBL['n']
periodic = MBL['periodic']
p        = MBL['periodic']
num_EV   = MBL['num_EV']

In [ ]:
# Two classes.
labels = ['Extended (Low W)', 'Localized (High W)']

In [ ]:
def sigmoid(x, x0, y0, b):
    y = 1 / (1 + np.exp(-b * (x - x0))) + y0
    return y

# Logit function is the inverse of sigmoid.
def logit(y, x0, y0, b):
    x = np.log((y - y0) / (1 - (y - y0))) / b + x0
    return x

print(sigmoid(0,0,0,1))
print(logit(0.5,0,0,1))

In [ ]:
# Remove y0 because it should be bounded/aligned with y = 0 and y = 1.
def sigmoid(x, x0, b):
    y = 1 / (1 + np.exp(-b * (x - x0))) # + y0
    return y

# Logit function is the inverse of sigmoid.
def logit(y, x0, b):
    x = np.log((y) / (1 - (y))) / b + x0
    return x


In [ ]:
def calc_probs(model, dataset):

    # Sample model predictions.
    result_images  = []
    result_targets = []
    result_Ws      = []
    result_preds   = []
    result_probs   = []

    model.eval()
    SM  = torch.nn.Softmax()
    LSM = torch.nn.LogSoftmax()
    dataloader = DataLoader(dataset, batch_size=200, shuffle=False)#, pin_memory=True)
    for batch in dataloader:

        images, targets, Ws = batch["image"], batch["label"], batch["W"]
        images  = images.to(device)
        outputs = model(images)
        images  = images.to('cpu')
        outputs = outputs.to('cpu')

        preds   = outputs.argmax(axis=1)
        Ps      = SM(outputs)
        shape   = images.shape
        result_images  = result_images  + images.reshape(shape[0], shape[2], shape[3]).tolist()
        result_targets = result_targets + targets.tolist()
        result_Ws      = result_Ws      + Ws.tolist()
        result_preds   = result_preds   + preds.tolist()
        result_probs   = result_probs   + Ps.tolist()

    result_Ws    = np.array(result_Ws)
    result_probs = np.array(result_probs)
    sorted_idx   = result_Ws.argsort()
    Ws = result_Ws[sorted_idx]
    Ps = result_probs[sorted_idx]

    # Compute mean and std.
    Ws_dict = OrderedDict()
    Ws_uniq = []
    Ps_mean = []
    Ps_std  = []
    # Ws is already sorted in `calc_probs()`.
    for W, P in zip(Ws, Ps[:,1]):
        if W not in Ws_dict:
            Ws_dict[W] = []
        Ws_dict[W].append(P)
    for (W, P) in Ws_dict.items():
        Ws_uniq.append(W)
        Ps_mean.append(np.mean(P))
        Ps_std.append(np.std(P, ddof=1))

    return Ws, Ps, np.array(Ws_uniq), np.array(Ps_mean), np.array(Ps_std)


In [ ]:
# data = calc_probs(model, model.dataset["val"])
# save_eval_valid(  data, model_version, L, n, periodic, num_EV)
# save_H_eval_valid(data, model_version, L, periodic)

In [ ]:
def plot_crossing(Ws, Ps, Ws_uniq, Ps_mean, Ps_std, L, n, periodic):

    labels = ['Extended (Low W)', 'Localized (High W)']

    # Plot probability P(Localized) against W.
    # 4:3 aspect ratio.
    fig, axes = plt.subplots(1, 1, figsize=(fig_w/dpi, fig_h/dpi), dpi=dpi, squeeze=False)

    fig.suptitle('Probability vs Disorder Strength ($L={}, n={}, {}$)'.format(L, n, 'periodic' if periodic else 'non-periodic'))
    axes[0,0].axhline(0.5, c='lightgrey', ls='--', label='$P=0.5$')

    # Plot averaged values with error bars.
    # markers, caps, bars = axes[0,0].errorbar(Ws_uniq, Ps_mean, Ps_std, ls=' ', marker='x', capsize=2, capthick=2, label='P(Localized) Mean')
    # Loop through bars and caps and set the alpha value
    # [bar.set_alpha(0.5) for bar in bars]
    # [cap.set_alpha(0.5) for cap in caps]
    
    # Actually, don't plot error bars. That's distracting.
    axes[0,0].plot(Ws_uniq, Ps_mean, ls=' ', marker='x', c='#0065bd', label='$E_W[P(Localized)]$')

    # Plot raw data.
    axes[0,0].plot(Ws, Ps, ls=' ', marker='.', c='#98c6ea', label='$P(Localized)$', alpha=0.05)
    axes[0,0].set_xlabel('Disorder strength $W$')
    axes[0,0].set_ylabel('Probability $P(Localized)$')

    # Curve fit a sigmoid using all data.
    # Fitting only the mean with `Ws_uniq` and `Ps_mean` gives identical results.
    # popt, pcov = curve_fit(sigmoid, Ws, Ps, p0=[3, 0, 2]) # Add bounds or initial values if it doesn't converge.
    popt, pcov = curve_fit(sigmoid, Ws, Ps, p0=[3, 2]) # Add bounds or initial values if it doesn't converge.
    # x0, y0, b = popt
    x0, b = popt
    x = np.linspace(0, 10, 100)
    y = sigmoid(x, *popt)
    axes[0,0].plot(x, y, ls='--', lw=2, c='#e37222', label='Fited sigmoid')
    axes[0,0].set_title('$\sigma(x) = 1 / (1 + Exp(-{:.4f} (x - {:.4f})))$'.format(b, x0), fontsize=10)
    # print('Fitted sigmoid function 1 / (1 + Exp(-{:.4f} (x - {:.4f}))) + {:.4f}'.format(b, x0, y0))
    print('Fitted sigmoid function 1 / (1 + Exp(-{:.4f} (x - {:.4f})))'.format(b, x0))

    W_c = logit(0.5, *popt)
    perr = np.sqrt(np.diag(pcov))
    print('Transition W_C is found to be at W = {:.4f} ± {:.4f}'.format(W_c, perr[0]))
    axes[0,0].axvline(W_c, c='r', ls='--', label='Fitted $W_c$')

    for axe in axes:
        for ax in axe:
            ax.tick_params(direction="in")
            ax.legend(loc='best')
            ax.set_xlim(0, 6)


In [ ]:
model_version = 1
# num_EV = 1

In [ ]:
valid_Ws, valid_Ps, valid_Ws_uniq, valid_Ps_mean, valid_Ps_std = load_eval_valid(  model_version, 14, 6, periodic, num_EV)
plot_crossing(valid_Ws, valid_Ps[:,1], valid_Ws_uniq, valid_Ps_mean, valid_Ps_std, 14, 6, periodic)

In [ ]:
valid_Ws, valid_Ps, valid_Ws_uniq, valid_Ps_mean, valid_Ps_std = load_eval_valid(  model_version, 12, 6, periodic, num_EV)
# valid_Ws, valid_Ps, valid_Ws_uniq, valid_Ps_mean, valid_Ps_std = load_H_eval_valid(model_version, L, periodic)

In [ ]:
plot_crossing(valid_Ws, valid_Ps[:,1], valid_Ws_uniq, valid_Ps_mean, valid_Ps_std, 12, 6, periodic)

In [ ]:
valid_Ws, valid_Ps, valid_Ws_uniq, valid_Ps_mean, valid_Ps_std = load_eval_valid(  model_version, 10, 6, periodic, num_EV)
plot_crossing(valid_Ws, valid_Ps[:,1], valid_Ws_uniq, valid_Ps_mean, valid_Ps_std, 10, 6, periodic)

In [ ]:
valid_Ws, valid_Ps, valid_Ws_uniq, valid_Ps_mean, valid_Ps_std = load_eval_valid(  model_version, 8, 6, periodic, num_EV)
plot_crossing(valid_Ws, valid_Ps[:,1], valid_Ws_uniq, valid_Ps_mean, valid_Ps_std, 8, 6, periodic)

In [ ]:
# Why L=13 n=5 k=1 is so skewed:
valid_Ws, valid_Ps, valid_Ws_uniq, valid_Ps_mean, valid_Ps_std = load_eval_valid(  model_version, 13, 5, periodic, 1)
plot_crossing(valid_Ws, valid_Ps[:,1], valid_Ws_uniq, valid_Ps_mean, valid_Ps_std, 13, 5, periodic)

In [ ]:
# Why L=14 n=6 k=1 is so skewed:
valid_Ws, valid_Ps, valid_Ws_uniq, valid_Ps_mean, valid_Ps_std = load_eval_valid(  model_version, 14, 6, False, 1)
plot_crossing(valid_Ws, valid_Ps[:,1], valid_Ws_uniq, valid_Ps_mean, valid_Ps_std, 14, 6, False)

In [ ]:
def test_slope():

    labels = ['Extended (Low W)', 'Localized (High W)']

    # Plot probability P(Localized) against W.
    # 4:3 aspect ratio.
    fig, axes = plt.subplots(1, 1, figsize=(fig_w/dpi, fig_h/dpi), dpi=dpi, squeeze=False)

    axes[0,0].axhline(0.5, c='lightgrey', ls='--', label='$P=0.5$')

    for b in [0.1, 1, 10]:
        x0 = 3
        x = np.linspace(0, 10, 100)
        y = sigmoid(x, 3, b)
        axes[0,0].plot(x, y, ls='--', lw=2, label='Fited sigmoid {}'.format(b))
        # print('Fitted sigmoid function 1 / (1 + Exp(-{:.4f} (x - {:.4f}))) + {:.4f}'.format(b, x0, y0))
        print('Fitted sigmoid function 1 / (1 + Exp(-{:.4f} (x - {:.4f})))'.format(b, x0))

    for axe in axes:
        for ax in axe:
            ax.tick_params(direction="in")
            ax.legend(loc='best')
            ax.set_xlim(0, 6)

test_slope()

## Batch process $W_c$

In [ ]:
def fit_Wc(Ws, Ps):
    """Curve fit a sigmoid using all data to obtain critical transition strength W_c.

    Fitting only the mean with `Ws_uniq` and `Ps_mean` gives identical results.
    """

    # popt, pcov = curve_fit(sigmoid, Ws, Ps, p0=[3, 0, 2]) # Add bounds or initial values if it doesn't converge.
    popt, pcov = curve_fit(sigmoid, Ws, Ps, p0=[3, 2]) # Add bounds or initial values if it doesn't converge.
    # x0, y0, b = popt
    x0, b = popt

    # Identifying tran
    # Wc = logit(0.5, *popt) # When y0 is considered. Otherwise W_c = x0.
    Wc = x0

    # Error terms.
    perr = np.sqrt(np.diag(pcov))
    We, be = perr
    
    return Wc, We, b, be


In [ ]:
def get_key(L, n, periodic, num_EV):
    """Too lazy to code a nested data structure."""
    return json.dumps([L, n, periodic, num_EV])#[1:-1]


In [ ]:
# Batch generate reduced density matrix.

# k = 5
J  = 1                      # Always = 1
# Ls = list(range(8,15,2))    # System sizes L.
Ls = list(range(8,15))      # System sizes L.
ps = [False, True]          # Periodic or not.
et = []                     # Execution time.
# num_EVs = [k]               # Number of eigenvalues near zero to save.
num_EVs = [1,5]             # Number of eigenvalues near zero to save.
model_version = 1
fit_result = {}

iter_count = 0
for L in Ls: # 7
    for n in range(1,9): # 8
        for periodic in ps: # 2
            for num_EV in num_EVs: # 2
                iter_count += 1
                # start_time = time.time()
                print('{} | Curve fitting W_c for L={:02d} | n={:d} | num_EV={} | periodic={: <5}...'.format(dt(), L, n, num_EV, str(periodic)), flush=True)
                try:
                    Ws, Ps, Ws_uniq, Ps_mean, Ps_std = load_eval_valid(  model_version, L, n, periodic, num_EV)
                    Wc, We, b, be = fit_Wc(Ws, Ps[:,1])
                    key = get_key(L, n, periodic, num_EV)
                    fit_result[key] = Wc, We, b, be
                except Exception as err:
                    print('{} | DATA NOT FOUND!'.format(dt()))
                    print(err)
                    key = get_key(L, n, periodic, num_EV)
                    Wc = np.nan
                    fit_result[key] = np.nan, np.nan, np.nan, np.nan # Return nan, s.t. matplotlib knows to skip.
                # exec_time = time.time() - start_time
                # et.append(exec_time)
                print('{} | Computed Wc = {:.4f}: L={:02d} | n={:d} | num_EV={} | periodic={: <5}.'.format(dt(), Wc, L, n, num_EV, str(periodic)), flush=True)
                # print('{} | Execution took {: 8.2f}s or {: 6.2f}min.'.format(dt(), exec_time, exec_time/60), flush=True)
                print(' ', flush=True)

print('Total number of iterations: {}'.format(iter_count)) # = 48... = 224
# if check_shutdown_signal():
#     break

In [ ]:
from matplotlib.colors import to_rgb, to_rgba
def get_rgba(color, alpha_arr):
    r, g, b = to_rgb(color)
    colors = [(r, g, b, alpha) for alpha in alpha_arr]
    return colors

# https://stackoverflow.com/a/39634143
def list_from_cycle(cycle):
    first = next(cycle)
    result = [first]
    for current in cycle:
        if current == first:
            break
        result.append(current)

    # Reset iterator state:
    for current in cycle:
        if current == result[-1]:
            break
    return result


In [ ]:
def plot_scaling_W(fit_result, all_data=False):

    labels = ['Extended (Low W)', 'Localized (High W)']

    # Plot Wc against L.
    # 4:3 aspect ratio.
    fig, axes = plt.subplots(3, 2, figsize=(fig_w/dpi*2, fig_h/dpi*3), dpi=dpi, squeeze=False, sharey='row')

    fig.suptitle('Scaling of Critical Disorder Strength $W_c$', fontsize=14)
    axes[0,0].set_title('Periodic boundary condition')
    axes[0,1].set_title('Open boundary condition')
    axes[0,1].yaxis.tick_right()
    axes[0,1].yaxis.set_label_position("right")
    axes[1,1].yaxis.tick_right()
    axes[1,1].yaxis.set_label_position("right")
    axes[2,1].yaxis.tick_right()
    axes[2,1].yaxis.set_label_position("right")
    
    axes[0,0].set_xlabel('System size $L$')
    axes[0,1].set_xlabel('System size $L$')
    axes[1,0].set_xlabel('Subsystem size $n$')
    axes[1,1].set_xlabel('Subsystem size $n$')
    axes[2,0].set_xlabel('$n / L$')
    axes[2,1].set_xlabel('$n / L$')
    axes[2,0].set_xlabel('$n log(n) + (L-n) log(L-n)$')
    axes[2,1].set_xlabel('$n log(n) + (L-n) log(L-n)$')

    if all_data:
        Ls = list(range(8,15))
        ns = list(range(1, 9))
    else:
        Ls = list(range(8,13))
        ns = list(range(1, 7))
    Ls_arr = np.array(Ls)
    ns_arr = np.array(ns)



    # ================================================================================
    # First plot `k = 1` data from Dataset 3.
    # ================================================================================

    k = 1

    for p in [False, True]:
        for n in ns:
            Wcs = []
            Wcs_err = []
            for L in Ls:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                Wcs.append(Wc)
                Wcs_err.append(We)
            # Errorbars are not shown because they are less than a pixel.
            Wcs = np.array(Wcs)
            nan_mask = np.isfinite(Wcs)
            # np.power(Ls_arr[nan_mask],5/4)
            if p:
                axes[0,0].plot(Ls_arr[nan_mask], Wcs[nan_mask], marker='x', ls='-', label='k={:d} n={:d}'.format(k,n))
            else:
                axes[0,1].plot(Ls_arr[nan_mask], Wcs[nan_mask], marker='+', ls='-', label='k={:d} n={:d}'.format(k,n))


    for p in [False, True]:
        for L in Ls:
            Wcs = []
            Wcs_err = []
            for n in ns:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                Wcs.append(Wc)
                Wcs_err.append(We)
            # Errorbars are not shown because they are less than a pixel.
            Wcs = np.array(Wcs)
            nan_mask = np.isfinite(Wcs)
            if p:
                axes[1,0].plot(ns_arr[nan_mask], Wcs[nan_mask], marker='x', ls='-', label='k={:d} L={:d}'.format(k,L))
            else:
                axes[1,1].plot(ns_arr[nan_mask], Wcs[nan_mask], marker='+', ls='-', label='k={:d} L={:d}'.format(k,L))


    for p in [False, True]:
        nL = []
        Wcs = []
        Wcs_err = []
        for i, L in enumerate(Ls):
            nL = []
            Wcs = []
            Wcs_err = []
            for n in ns:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                # nL.append(n / L)
                # nL.append(1 / (n / L))
                # nL.append((n / (L - n)))
                nL.append(n * np.log(n) + (L-n) * np.log(L-n))
                Wcs.append(Wc)
                Wcs_err.append(We)
            # Errorbars are not shown because they are less than a pixel.
            nL  = np.array(nL)
            Wcs = np.array(Wcs)
            nan_mask = np.isfinite(Wcs)
            if p:
                # Applying alpha to a list of colors somehow doesn't work.
                # cs = get_rgba(list_from_cycle(axes[2,0]._get_lines.prop_cycler)[0]['color'], (np.array(ns)+1)/L)
                # print(len(nL), len(ns), len(cs))
                # cs=np.array(cs).reshape(-1,4)
                axes[2,0].plot(nL[nan_mask], Wcs[nan_mask], marker='x', ls='-', label='k={:d} L={:d}'.format(k,L))
            else:
                axes[2,1].plot(nL[nan_mask], Wcs[nan_mask], marker='+', ls='-', label='k={:d} L={:d}'.format(k,L))
        # if p:
        #     axes[2,0].plot(nL, Wcs, marker='x', ls='' )
        # else:
        #     axes[2,1].plot(nL, Wcs, marker='+', ls='')



    # ================================================================================
    # Then plot `k = 5` data from Dataset 2, at half opacity.
    # ================================================================================

    for row, axe in enumerate(axes):
        for col, ax in enumerate(axe):
            ax.set_prop_cycle(None)

    k = 5

    for p in [False, True]:
        for n in ns:
            Wcs = []
            Wcs_err = []
            for L in Ls:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                Wcs.append(Wc)
                Wcs_err.append(We)
            # Errorbars are not shown because they are less than a pixel.
            Wcs = np.array(Wcs)
            nan_mask = np.isfinite(Wcs)
            if p:
                axes[0,0].plot(Ls_arr[nan_mask], Wcs[nan_mask], marker='x', ls='--', alpha=0.5, label='k={:d} n={:d}'.format(k,n))
            else:
                axes[0,1].plot(Ls_arr[nan_mask], Wcs[nan_mask], marker='+', ls='--', alpha=0.5, label='k={:d} n={:d}'.format(k,n))


    for p in [False, True]:
        for L in Ls:
            Wcs = []
            Wcs_err = []
            for n in ns:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                Wcs.append(Wc)
                Wcs_err.append(We)
            # Errorbars are not shown because they are less than a pixel.
            Wcs = np.array(Wcs)
            nan_mask = np.isfinite(Wcs)
            if p:
                axes[1,0].plot(ns_arr[nan_mask], Wcs[nan_mask], marker='x', ls='--', alpha=0.5, label='k={:d} L={:d}'.format(k,L))
            else:
                axes[1,1].plot(ns_arr[nan_mask], Wcs[nan_mask], marker='+', ls='--', alpha=0.5, label='k={:d} L={:d}'.format(k,L))


    for p in [False, True]:
        nL = []
        Wcs = []
        Wcs_err = []
        for i, L in enumerate(Ls):
            nL = []
            Wcs = []
            Wcs_err = []
            for n in ns:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                # nL.append(n / L)
                # nL.append(1 / (n / L))
                # nL.append((n / (L - n)))
                nL.append(n * np.log(n) + (L-n) * np.log(L-n))
                Wcs.append(Wc)
                Wcs_err.append(We)
            # Errorbars are not shown because they are less than a pixel.
            nL  = np.array(nL)
            Wcs = np.array(Wcs)
            nan_mask = np.isfinite(Wcs)
            if p:
                # Applying alpha to a list of colors somehow doesn't work.
                # cs = get_rgba(list_from_cycle(axes[2,0]._get_lines.prop_cycler)[0]['color'], (np.array(ns)+1)/L)
                # print(len(nL), len(ns), len(cs))
                # cs=np.array(cs).reshape(-1,4)
                axes[2,0].plot(nL[nan_mask], Wcs[nan_mask], marker='x', ls='--', alpha=0.5, label='k={:d} L={:d}'.format(k,L))
            else:
                axes[2,1].plot(nL[nan_mask], Wcs[nan_mask], marker='+', ls='--', alpha=0.5, label='k={:d} L={:d}'.format(k,L))
        # if p:
        #     axes[2,0].plot(nL, Wcs, marker='x', ls='' , alpha=0.5)
        # else:
        #     axes[2,1].plot(nL, Wcs, marker='+', ls='', alpha=0.5)



    # ================================================================================
    # Configure axes settings.
    # ================================================================================

    for row, axe in enumerate(axes):
        for col, ax in enumerate(axe):
            if row == 0:
                # ax.legend(loc=(0.03, 0.58), ncol=3, framealpha=0.5)#, bbox_to_anchor=(1, 0.05))
                if col == 0:
                    if all_data:
                        ax.legend(loc='lower left', ncol=3, framealpha=0.5)
                    else:
                        ax.legend(loc='lower right', ncol=3, framealpha=0.5, bbox_to_anchor=(0.995, 0.58))
                if col == 1:
                    if all_data:
                        ax.legend(loc='lower left', ncol=3, framealpha=0.5)
                    else:
                        ax.legend(loc='lower right', ncol=3, framealpha=0.5, bbox_to_anchor=(0.995, 0.48))
            if row == 1:
                if all_data:
                    ax.legend(loc='lower left', ncol=2, framealpha=0.5)
                else:
                    ax.legend(loc='upper right', ncol=2, framealpha=0.5)
            if row == 2:
                if all_data:
                    ax.legend(loc='lower left', ncol=2, framealpha=0.5)
                else:
                    ax.legend(loc='lower right', ncol=2, framealpha=0.5)

            ax.tick_params(direction="in")
            ax.set_ylabel('Critical disorder strength $W_c$')
            ax.xaxis.get_major_locator().set_params(integer=True)

    fig.tight_layout()
    # fig.subplots_adjust(top=0.88)
    fig.subplots_adjust(top=0.94)

In [ ]:
def plot_scaling_b(fit_result, all_data=False):

    labels = ['Extended (Low W)', 'Localized (High W)']

    # Plot Wc against L.
    # 4:3 aspect ratio.
    fig, axes = plt.subplots(3, 2, figsize=(fig_w/dpi*2, fig_h/dpi*3), dpi=dpi, squeeze=False, sharey='row')

    fig.suptitle('Scaling of Transition Steepness $b$', fontsize=14)
    axes[0,0].set_title('Periodic boundary condition')
    axes[0,1].set_title('Open boundary condition')
    axes[0,1].yaxis.tick_right()
    axes[0,1].yaxis.set_label_position("right")
    axes[1,1].yaxis.tick_right()
    axes[1,1].yaxis.set_label_position("right")
    axes[2,1].yaxis.tick_right()
    axes[2,1].yaxis.set_label_position("right")
    
    axes[0,0].set_xlabel('System size $L$')
    axes[0,1].set_xlabel('System size $L$')
    axes[1,0].set_xlabel('Subsystem size $n$')
    axes[1,1].set_xlabel('Subsystem size $n$')
    axes[2,0].set_xlabel('$n / L$')
    axes[2,1].set_xlabel('$n / L$')
    axes[2,0].set_xlabel('$n log(n) + (L-n) log(L-n)$')
    axes[2,1].set_xlabel('$n log(n) + (L-n) log(L-n)$')

    if all_data:
        Ls = list(range(8,15))
        ns = list(range(1, 9))
    else:
        Ls = list(range(8,13))
        ns = list(range(1, 7))
    Ls_arr = np.array(Ls)
    ns_arr = np.array(ns)



    # ================================================================================
    # First plot `k = 1` data from Dataset 3.
    # ================================================================================

    k = 1

    for p in [False, True]:
        for n in ns:
            bcs = []
            bcs_err = []
            for L in Ls:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                bcs.append(b)
                bcs_err.append(be)
            # Errorbars are not shown because they are less than a pixel.
            bcs = np.array(bcs)
            nan_mask = np.isfinite(bcs)
            if p:
                axes[0,0].plot(Ls_arr[nan_mask], bcs[nan_mask], marker='x', ls='-', label='k={:d} n={:d}'.format(k,n))
            else:
                axes[0,1].plot(Ls_arr[nan_mask], bcs[nan_mask], marker='+', ls='-', label='k={:d} n={:d}'.format(k,n))


    for p in [False, True]:
        for L in Ls:
            bcs = []
            bcs_err = []
            for n in ns:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                bcs.append(b)
                bcs_err.append(be)
            # Errorbars are not shown because they are less than a pixel.
            bcs = np.array(bcs)
            nan_mask = np.isfinite(bcs)
            if p:
                axes[1,0].plot(ns_arr[nan_mask], bcs[nan_mask], marker='x', ls='-', label='k={:d} L={:d}'.format(k,L))
            else:
                axes[1,1].plot(ns_arr[nan_mask], bcs[nan_mask], marker='+', ls='-', label='k={:d} L={:d}'.format(k,L))


    for p in [False, True]:
        nL = []
        bcs = []
        bcs_err = []
        for L in Ls:
            nL = []
            bcs = []
            bcs_err = []
            for n in ns:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                # nL.append(n / L)
                # nL.append(1 / (n / L))
                # nL.append((n / (L - n)))
                nL.append(n * np.log(n) + (L-n) * np.log(L-n))
                bcs.append(b)
                bcs_err.append(be)
            # Errorbars are not shown because they are less than a pixel.
            nL  = np.array(nL)
            bcs = np.array(bcs)
            nan_mask = np.isfinite(bcs)
            if p:
                axes[2,0].plot(nL[nan_mask], bcs[nan_mask], marker='x', ls='-', label='k={:d} L={:d}'.format(k,L))
            else:
                axes[2,1].plot(nL[nan_mask], bcs[nan_mask], marker='+', ls='-', label='k={:d} L={:d}'.format(k,L))
        # if p:
        #     axes[2,0].plot(nL, bcs, marker='x', ls='' )
        # else:
        #     axes[2,1].plot(nL, bcs, marker='+', ls='')



    # ================================================================================
    # Then plot `k = 5` data from Dataset 2, at half opacity.
    # ================================================================================

    for row, axe in enumerate(axes):
        for col, ax in enumerate(axe):
            ax.set_prop_cycle(None)

    k = 5

    for p in [False, True]:
        for n in ns:
            bcs = []
            bcs_err = []
            for L in Ls:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                bcs.append(b)
                bcs_err.append(be)
            # Errorbars are not shown because they are less than a pixel.
            bcs = np.array(bcs)
            nan_mask = np.isfinite(bcs)
            if p:
                axes[0,0].plot(Ls_arr[nan_mask], bcs[nan_mask], marker='x', ls='--', alpha=0.5, label='k={:d} n={:d}'.format(k,n))
            else:
                axes[0,1].plot(Ls_arr[nan_mask], bcs[nan_mask], marker='+', ls='--', alpha=0.5, label='k={:d} n={:d}'.format(k,n))


    for p in [False, True]:
        for L in Ls:
            bcs = []
            bcs_err = []
            for n in ns:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                bcs.append(b)
                bcs_err.append(be)
            # Errorbars are not shown because they are less than a pixel.
            bcs = np.array(bcs)
            nan_mask = np.isfinite(bcs)
            if p:
                axes[1,0].plot(ns_arr[nan_mask], bcs[nan_mask], marker='x', ls='--', alpha=0.5, label='k={:d} L={:d}'.format(k,L))
            else:
                axes[1,1].plot(ns_arr[nan_mask], bcs[nan_mask], marker='+', ls='--', alpha=0.5, label='k={:d} L={:d}'.format(k,L))


    for p in [False, True]:
        nL = []
        bcs = []
        bcs_err = []
        for L in Ls:
            nL = []
            bcs = []
            bcs_err = []
            for n in ns:
                key = get_key(L, n, p, k)
                Wc, We, b, be = fit_result[key]
                # nL.append(n / L)
                # nL.append(1 / (n / L))
                # nL.append((n / (L - n)))
                nL.append(n * np.log(n) + (L-n) * np.log(L-n))
                bcs.append(b)
                bcs_err.append(be)
            # Errorbars are not shown because they are less than a pixel.
            nL  = np.array(nL)
            bcs = np.array(bcs)
            nan_mask = np.isfinite(bcs)
            if p:
                axes[2,0].plot(nL[nan_mask], bcs[nan_mask], marker='x', ls='--', alpha=0.5, label='k={:d} L={:d}'.format(k,L))
            else:
                axes[2,1].plot(nL[nan_mask], bcs[nan_mask], marker='+', ls='--', alpha=0.5, label='k={:d} L={:d}'.format(k,L))
        # if p:
        #     axes[2,0].plot(nL, bcs, marker='x', ls='' , alpha=0.5)
        # else:
        #     axes[2,1].plot(nL, bcs, marker='+', ls='', alpha=0.5)



    # ================================================================================
    # Configure axes settings.
    # ================================================================================

    for row, axe in enumerate(axes):
        for col, ax in enumerate(axe):
            if row == 0:
                if all_data:
                    ax.legend(loc='lower left', ncol=2, framealpha=0.5)
                else:
                    ax.legend(loc='lower right', ncol=2, framealpha=0.5)#, bbox_to_anchor=(1, 0.05))

                # Set y scale to exponential
                # exp = lambda x: np.power(10, x)
                # log = lambda x: np.log(x)
                # ax.set_yscale('function', functions=(exp, log))
                # ax.set_yscale('log')
            if row == 1:
                ax.legend(loc='lower right', ncol=2, framealpha=0.5)
                # ax.set_xscale('log')
            if row == 2:
                ax.legend(loc='lower left', ncol=2, framealpha=0.5)

            ax.tick_params(direction="in")
            ax.set_ylabel('Transition steepness $b$')
            ax.xaxis.get_major_locator().set_params(integer=True)
            # ax.set_ylim(0)

    fig.tight_layout()
    # fig.subplots_adjust(top=0.88)
    fig.subplots_adjust(top=0.94)

In [ ]:
plot_scaling_W(fit_result)

In [ ]:
plot_scaling_b(fit_result)

In [ ]:
plot_scaling_W(fit_result, True)

In [ ]:
plot_scaling_b(fit_result, True)

## Direct Hamiltonian
$L=10, periodic$

In [ ]:
base_sample = 10000 # Samples per training W.
rand_sample = 50
Ws_train = np.random.randint(0,     2, size=(2 * base_sample,))
Ws_train = (Ws_train * 7.5) + 0.5 # i.e. Ws are 0.5 and 8.0.
Ws_valid = np.random.uniform(0.1, 5.9, size=(2 * base_sample // rand_sample,))
Ws_valid = (Ws_valid.reshape(-1, 1) * np.ones((1,50))).flatten()
print(Ws_train.shape)
print(Ws_valid.shape)

In [ ]:
MBL = {
    "obj_name": 'H',
    "L": 6,
    "periodic": True,
    "Ws_train": Ws_train,
    "Ws_valid": Ws_valid,
}
obj_name = MBL['obj_name']
L        = MBL['L']
periodic = MBL['periodic']

In [ ]:
from MBL_H_dataset import MBLHDataset

train_H_dataset = MBLHDataset(
    MBL_params=MBL,
    train=True,
    transform=transforms.ToTensor(),
    # transform=transforms.Compose([
    #     transforms.ToPILImage(),
    #     transforms.ToTensor()
    # ]),
)
valid_H_dataset = MBLHDataset(
    MBL_params=MBL,
    train=False,
    transform=transforms.ToTensor(),
    # transform=transforms.Compose([
    #     transforms.ToPILImage(),
    #     transforms.ToTensor()
    # ]),
)

print('Number of training samples  :', len(train_H_dataset))
print('Number of validation samples:', len(valid_H_dataset))

In [ ]:
def visualize_H(dataset):
    
    num = 4

    sample_idx = np.random.randint(0, len(dataset), size=num*num)

    # Square image.
    fig, axes = plt.subplots(num, num, figsize=(fig_w/dpi,fig_w/dpi), dpi=dpi, squeeze=False)

    samples_ext = []
    samples_loc = []
    for i in range(len(dataset)):
        data = dataset[i]
        if data['W'] == 0.5 and len(samples_ext) < num*num//2:
            samples_ext.append((data['image'][0,:,:].numpy()))
        if data['W'] == 8.0 and len(samples_loc) < num*num//2:
            samples_loc.append((data['image'][0,:,:].numpy()))
        if len(samples_ext) >= num*num//2 and len(samples_loc) >= num*num//2:
            break

    i_ext = 0
    i_loc = 0
    for i in range(num * num):
        if i // num < 2:
            # np.fill_diagonal(samples_ext[i_ext], 0)
            axes[i%num,i//num].imshow(samples_ext[i_ext])
            axes[i%num,i//num].annotate('W={:3.1f}'.format(0.5), (0.25,0.1), xycoords='axes fraction', ha='center', color='w')
            i_ext += 1
        else:
            # np.fill_diagonal(samples_loc[i_loc], 0)
            axes[i%num,i//num].imshow(samples_loc[i_loc])
            axes[i%num,i//num].annotate('W={:3.1f}'.format(8.0), (0.25,0.1), xycoords='axes fraction', ha='center', color='w')
            i_loc += 1

    for axe in axes:
        for ax in axe:
            # ax.legend(loc='best')
            ax.xaxis.set_ticklabels([])
            ax.yaxis.set_ticklabels([])
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False)

    fig.tight_layout()

In [ ]:
print('Visualize Hamiltonian training data:')
visualize_H(train_H_dataset)

In [ ]:
# from MBL_H_model import MBLHModel
# model_version = 1
# model_H = load_H_model('model_v{}.pkl.gz'.format(model_version), L=10, periodic=True)

In [ ]:
def plot_H_crossing(Ws, Ps, Ws_uniq, Ps_mean, Ps_std, L, periodic):

    labels = ['Extended (Low W)', 'Localized (High W)']

    # Plot probability P(Localized) against W.
    # 4:3 aspect ratio.
    fig, axes = plt.subplots(1, 1, figsize=(fig_w/dpi, fig_h/dpi), dpi=dpi, squeeze=False)

    fig.suptitle('Probability vs Disorder Strength ($L={}, {}$)'.format(L, 'periodic' if periodic else 'non-periodic'))

    axes[0,0].axhline(0.5, c='lightgrey', ls='--', label='$P=0.5$')

    # Plot averaged values with error bars.
    # markers, caps, bars = axes[0,0].errorbar(Ws_uniq, Ps_mean, Ps_std, ls=' ', marker='x', capsize=2, capthick=2, label='P(Localized) Mean')
    # Loop through bars and caps and set the alpha value
    # [bar.set_alpha(0.5) for bar in bars]
    # [cap.set_alpha(0.5) for cap in caps]
    
    # Actually, don't plot error bars. That's distracting.
    axes[0,0].plot(Ws_uniq, Ps_mean, ls=' ', marker='x', c='#0065bd', label='$E_W[P(Localized)]$')

    # Plot raw data.
    axes[0,0].plot(Ws, Ps,   ls=' ', marker='.', c='#98c6ea', label='$P(Localized)$', alpha=0.05)
    axes[0,0].set_xlabel('Disorder strength $W$')
    axes[0,0].set_ylabel('Probability $P(Localized)$')

    # Curve fit a sigmoid using all data.
    # Fitting only the mean with `Ws_uniq` and `Ps_mean` gives identical results.
    # popt, pcov = curve_fit(sigmoid, Ws, Ps, p0=[3, 0, 2]) # Add bounds or initial values if it doesn't converge.
    popt, pcov = curve_fit(sigmoid, Ws, Ps, p0=[3, 2]) # Add bounds or initial values if it doesn't converge.
    # x0, y0, b = popt
    x0, b = popt
    x = np.linspace(0, 10, 100)
    y = sigmoid(x, *popt)
    axes[0,0].plot(x, y, ls='--', lw=2, c='#e37222', label='Fited sigmoid')
    axes[0,0].set_title('$\sigma(x) = 1 / (1 + Exp(-{:.4f} (x - {:.4f})))$'.format(b, x0), fontsize=10)
    # print('Fitted sigmoid function 1 / (1 + Exp(-{:.4f} (x - {:.4f}))) + {:.4f}'.format(b, x0, y0))
    print('Fitted sigmoid function 1 / (1 + Exp(-{:.4f} (x - {:.4f})))'.format(b, x0))

    W_c = logit(0.5, *popt)
    perr = np.sqrt(np.diag(pcov))
    print('Transition W_C is found to be at W = {:.4f} ± {:.4f}'.format(W_c, perr[0]))
    axes[0,0].axvline(W_c, c='r',         ls='--', label='Fitted $W_c$')

    for axe in axes:
        for ax in axe:
            ax.tick_params(direction="in")
            ax.legend(loc='best')
            ax.set_xlim(0, 6)


In [ ]:
Ws, Ps, Ws_uniq, Ps_mean, Ps_std = load_H_eval_valid(  model_version, 10, periodic)
# Ws, Ps, Ws_uniq, Ps_mean, Ps_std = load_H_eval_valid(model_version, L, periodic)
plot_H_crossing(Ws, Ps[:,1], Ws_uniq, Ps_mean, Ps_std, 10, periodic)

In [ ]:
# del model_H

## Testing tensordot

In [ ]:
a = np.random.rand((200))
b = np.random.rand((200))
u = a + 1j * b
a = np.random.rand((200))
b = np.random.rand((200))
v = a + 1j * b

In [ ]:
u1 = np.tensordot(u, u, axes=[[0],[0]])
u2 = np.tensordot(u.conj(), u, axes=[[0],[0]])
u3 = np.tensordot(u, u.conj(), axes=[[0],[0]])
print(u1)
print(u2)
print(u3)

In [ ]:
uv1 = np.outer(u, v)
uv2 = np.outer(u.conj(), v)
uv3 = np.outer(u, v.conj())

In [ ]:
np.allclose(uv2.conj(), uv3)